In [1]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
from FinMind.data import DataLoader
import matplotlib.pyplot as plt
from neuralprophet import NeuralProphet
from statsmodels.tsa.stattools import pacf
import numpy as np

api = DataLoader()

d:\Practice\stock_trade\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [2]:
def best_pacf_lag(data, max_lags=30):
    pacf_values = pacf(data, nlags=max_lags, method="ywadjusted")
    conf_int = 1.96 / np.sqrt(len(data))  # 95% confidence threshold

    # Find first lag where PACF falls within confidence bounds
    for lag in range(1, len(pacf_values)):
        if abs(pacf_values[lag]) < conf_int:
            return lag - 1

    return max_lags

In [5]:
today = datetime.today()
start_date = (today - relativedelta(years=5)).strftime("%Y-%m-%d")
end_date = today.strftime("%Y-%m-%d")

ticker='2603'
df = api.taiwan_stock_daily(
    stock_id=ticker, start_date=start_date, end_date=end_date
)
df = df[["date", "close"]]
df.columns = ["ds", "y"]
df["ds"] = pd.to_datetime(df["ds"])

print('lags', best_pacf_lag(df["y"]))
# 訓練模型
m = NeuralProphet(
    n_changepoints=0,
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False,
    n_lags=best_pacf_lag(df["y"])
)
m.fit(df, epochs=100, batch_size=32, early_stopping=True)
forecast = m.predict(df)
m.plot_components(forecast)

2025-11-24 10:20:45.657 | INFO     | FinMind.data.finmind_api:get_data:171 - download Dataset.TaiwanStockPrice, data_id: 2603
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - d:\Practice\stock_trade\.venv\Lib\site-packages\neuralprophet\df_utils.py:464: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


WARNING - (py.warnings._showwarnmsg) - d:\Practice\stock_trade\.venv\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) -

lags 1
Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (41) is too small than the required number                     for the learning rate finder (228). The results might not be optimal.
Finding best initial lr: 100%|██████████| 228/228 [00:01<00:00, 154.35it/s]


Training: |          | 0/? [00:21<?, ?it/s, v_num=6, train_loss=0.000692, reg_loss=0.000, MAE=3.160, RMSE=4.610, Loss=0.000691, RegLoss=0.000]

WARNING - (py.warnings._showwarnmsg) - d:\Practice\stock_trade\.venv\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Major frequency B corresponds to 96.275% of the data.
WARNING - (py.warnings._showwarnmsg) - d:\Practice\stock_trade\.venv\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


WARNING - (py.warnings._showwarnmsg) - d:\Practice\stock_trade\.venv\Lib\site-packages\neuralprophet\df_utils.py:1152: FutureWarning:

Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.


INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - B
WARNING - (py.warnings._showwarnmsg) - d:\


Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 145.57it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
ERROR - (NP.plotly.plot_components) - plotly-resampler is not installed. Please install it to use the resampler.


WARNING - (py.warnings._showwarnmsg) - d:\Practice\stock_trade\.venv\Lib\site-packages\neuralprophet\plot_forecast_plotly.py:410: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result


WARNING - (py.warnings._showwarnmsg) - d:\Practice\stock_trade\.venv\Lib\site-packages\neuralprophet\plot_forecast_plotly.py:559: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result


